# Summary
Process NBI data for use with classification machine learning. Numeric and categorical features are exported separately.

- Filters:
    - structures in VA
    - exclude culverts
    - 10-year window
    - no reconstructed or improved rating bridges after feature input year

# Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Data

In [2]:
# feature data year
feat_year = 2007
# year for target outcome
tar_year = 2017

In [3]:
local_path = 'data/VA/'
feat_file = 'VA07.txt'
tar_file = 'VA17.txt'

In [4]:
# load feature file
df_feat_raw = pd.read_csv(local_path + feat_file)
df_feat_raw.head()

/home/kevcon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,7,17,36,37,38,39,41,44,61,63,66,67,68,69,70,75,76,77,79,80,86,87,88,102,106,107,108,109,113,114,119,120,121,123,124,130,131,133) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,PROJ_NO,PROJ_SUFFIX,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE
0,51,1VA0069,1,8,8.0,00000,0,05,710.0,51087.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,12.0,1
1,51,1VA0075,1,8,0.0,00000,0,00,199.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,93.0,0
2,51,1VA0079,1,8,8.0,00000,0,05,710.0,51087.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,76.7,0
3,51,1VA0158,1,8,0.0,00000,0,00,99.0,21008.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,13.5,1
4,51,1VA0159,1,8,0.0,00000,0,00,99.0,21008.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,89.2,0


In [5]:
# load target file
df_tar_raw = pd.read_csv(local_path + tar_file)
df_tar_raw.head()

/home/kevcon/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,71,73,75,98) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING,STATUS_NO_10YR_RULE,CAT10,CAT23,CAT29
0,51,1VA0075,1,8,0,00000,0,0,199,0,...,NaN,NaN,NaN,0,NaN,93.0,0,G,7,558.72
1,51,1VA0158,1,8,0,00000,0,0,99,21008,...,NaN,NaN,NaN,1,NaN,13.0,1,P,4,877.80
2,51,1VA0159,1,8,0,00000,0,0,99,21008,...,NaN,NaN,NaN,0,NaN,90.2,0,F,6,186.24
3,51,1VA0174,1,8,0,00000,0,0,740,710,...,NaN,NaN,NaN,0,NaN,99.0,0,G,7,141.52
4,51,1VA0257,1,8,0,00000,0,0,810,51128,...,NaN,NaN,NaN,0,NaN,100.0,0,G,7,1314.87


## Assign

In [6]:
df_feat = df_feat_raw.copy()
df_tar = df_tar_raw.copy()

# Filter

## Filter out culverts

In [7]:
# remove observations coded with 19 (culvert) in item 43B
df_feat = df_feat[df_feat['STRUCTURE_TYPE_043B'] != 19]
df_tar = df_tar[df_tar['STRUCTURE_TYPE_043B'] != 19]

## Filter out duplicate records

In [8]:
# remove non-1 codes for item 5A (record type)
df_feat = df_feat[df_feat['RECORD_TYPE_005A'] == 1]
df_tar = df_tar[df_tar['RECORD_TYPE_005A'] == 1]

# Merge

In [9]:
# specify target columns to merge
tar_cols = [
    'STATE_CODE_001', 
    'STRUCTURE_NUMBER_008',
    'YEAR_RECONSTRUCTED_106',
    'SUFFICIENCY_RATING'
]

In [10]:
# merge feature and target dataframes
# retains only values existing in feature df
df_bridges = pd.merge(
    df_feat, 
    df_tar[tar_cols], 
    how='left', 
    on=['STATE_CODE_001', 'STRUCTURE_NUMBER_008'], 
    suffixes=('_feat_yr', '_tar_yr')
)
df_bridges.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING_feat_yr,STATUS_NO_10YR_RULE,YEAR_RECONSTRUCTED_106_tar_yr,SUFFICIENCY_RATING_tar_yr
0,51,1VA0069,1,8,8.0,00000,0,05,710.0,51087.0,...,NaN,NaN,NaN,NaN,1,NaN,12.0,1,NaN,NaN
1,51,1VA0075,1,8,0.0,00000,0,00,199.0,0.0,...,NaN,NaN,NaN,NaN,0,NaN,93.0,0,NaN,93.0
2,51,1VA0079,1,8,8.0,00000,0,05,710.0,51087.0,...,NaN,NaN,NaN,NaN,0,NaN,76.7,0,NaN,NaN
3,51,1VA0158,1,8,0.0,00000,0,00,99.0,21008.0,...,NaN,NaN,NaN,NaN,1,NaN,13.5,1,1987.0,13.0
4,51,1VA0159,1,8,0.0,00000,0,00,99.0,21008.0,...,NaN,NaN,NaN,NaN,0,NaN,89.2,0,NaN,90.2


In [11]:
# remove observations with NaN for target sufficiency rating
df_bridges.dropna(subset=['SUFFICIENCY_RATING_tar_yr'], inplace=True)

## Filter Bridges

In [12]:
# remove bridges in good and poor condition in feature year
df_bridges = df_bridges[(df_bridges['SUFFICIENCY_RATING_feat_yr'] < 80) & 
                        (df_bridges['SUFFICIENCY_RATING_feat_yr'] >= 50)]

In [13]:
# remove bridges improved prior to target year
df_bridges = df_bridges[df_bridges['SUFFICIENCY_RATING_tar_yr'] <= df_bridges['SUFFICIENCY_RATING_feat_yr']]

In [14]:
# remove bridges reconstructed after feature dataset year
df_bridges = df_bridges[df_bridges['YEAR_RECONSTRUCTED_106_tar_yr'] <= feat_year].copy()
df_bridges.drop(columns='YEAR_RECONSTRUCTED_106_tar_yr', inplace=True)
df_bridges.rename(columns={'YEAR_RECONSTRUCTED_106_feat_yr': 'YEAR_RECONSTRUCTED_106'}, inplace=True)

In [15]:
# drop any remaining duplicate records
df_bridges.drop_duplicates(subset=['STATE_CODE_001', 'STRUCTURE_NUMBER_008'], inplace=True)

## Replace Year with Age
For use in modeling, year values are converted to years from feature year

In [16]:
# years since construction
df_bridges['YEAR_BUILT_027'] = feat_year - df_bridges['YEAR_BUILT_027']
# rename column
df_bridges.rename(columns={'YEAR_BUILT_027': 'AGE'}, inplace=True)

In [17]:
# years since reconstruction
df_bridges['YEAR_RECONSTRUCTED_106'] = df_bridges.apply(
    lambda x: np.where(x['YEAR_RECONSTRUCTED_106'] == 0, 
                       x['AGE'], 
                       feat_year - x['YEAR_RECONSTRUCTED_106']), 
    axis=1)
# rename column
df_bridges.rename(columns={'YEAR_RECONSTRUCTED_106': 'RECON_AGE'}, inplace=True)

In [18]:
df_bridges.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,RECORD_TYPE_005A,ROUTE_PREFIX_005B,SERVICE_LEVEL_005C,ROUTE_NUMBER_005D,DIRECTION_005E,HIGHWAY_DISTRICT_002,COUNTY_CODE_003,PLACE_CODE_004,...,PROJ_SUFFIX,NBI_TYPE_OF_IMP,DTL_TYPE_OF_IMP,SPECIAL_CODE,STEP_CODE,STATUS_WITH_10YR_RULE,SUFFICIENCY_ASTERC,SUFFICIENCY_RATING_feat_yr,STATUS_NO_10YR_RULE,SUFFICIENCY_RATING_tar_yr
11,51,1VA3701,1,8,8.0,00000,0,05,810.0,82000.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,72.7,0,72.7
12,51,1VA591A,1,8,8.0,00000,0,00,810.0,51128.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,53.0,0,52.3
13,51,2VA00AA,1,8,1.0,00000,0,00,179.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,69.2,0,69.2
14,51,2VA00CC,1,8,1.0,00000,0,00,179.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,73.7,0,63.6
15,51,2VA00DD,1,8,1.0,00000,0,00,153.0,0.0,...,NaN,NaN,NaN,NaN,NaN,0,NaN,79.1,0,79.1


# Feature Data

In [19]:
# id columns
id_cols = [
    'STATE_CODE_001', 
    'STRUCTURE_NUMBER_008',
    'COUNTY_CODE_003',
    'FEATURES_DESC_006A',
    'FACILITY_CARRIED_007',
    'LOCATION_009',
    'LAT_016',
    'LONG_017'
]

In [20]:
# numeric features
num_cols = [
    'MIN_VERT_CLR_010',  
    'DETOUR_KILOS_019', 
    'AGE', # engineered feature
    'TRAFFIC_LANES_ON_028A', 
    'TRAFFIC_LANES_UND_028B',
    'ADT_029',
    'APPR_WIDTH_MT_032',
    'DEGREES_SKEW_034',
    'NAV_VERT_CLR_MT_039', 
    'NAV_HORR_CLR_MT_040',
    'MAIN_UNIT_SPANS_045', 
    'APPR_SPANS_046',
    'HORR_CLR_MT_047', 
    'MAX_SPAN_LEN_MT_048', 
    'STRUCTURE_LEN_MT_049',
    'LEFT_CURB_MT_050A', 
    'RIGHT_CURB_MT_050B', 
    'ROADWAY_WIDTH_MT_051',
    'DECK_WIDTH_MT_052', 
    'VERT_CLR_OVER_MT_053', 
    'VERT_CLR_UND_054B', 
    'LAT_UND_MT_055B',
    'LEFT_LAT_UND_MT_056',
    'RECON_AGE', # engineered feature
    'PERCENT_ADT_TRUCK_109',
    'SUFFICIENCY_RATING_feat_yr'
]

In [21]:
# categorical features
cat_cols = [
    'TOLL_020',
    'MAINTENANCE_021',
    'FUNCTIONAL_CLASS_026',
    'DESIGN_LOAD_031',
    'MEDIAN_CODE_033', 
    'STRUCTURE_FLARED_035',
    'RAILINGS_036A',
    'TRANSITIONS_036B', 
    'APPR_RAIL_036C',
    'APPR_RAIL_END_036D', 
    'HISTORY_037', 
    'NAVIGATION_038',
    'OPEN_CLOSED_POSTED_041',
    'SERVICE_ON_042A', 
    'SERVICE_UND_042B', 
    'STRUCTURE_KIND_043A',
    'STRUCTURE_TYPE_043B', 
    'APPR_KIND_044A', 
    'APPR_TYPE_044B',
    'DECK_STRUCTURE_TYPE_107', 
    'SURFACE_TYPE_108A', 
    'MEMBRANE_TYPE_108B',
    'DECK_PROTECTION_108C',
    'DECK_COND_058',
    'SUPERSTRUCTURE_COND_059',
    'SUBSTRUCTURE_COND_060'
]

In [22]:
# reduce to specified, ordered columns
df_bridges = df_bridges[id_cols + num_cols + cat_cols + ['SUFFICIENCY_RATING_tar_yr']]
df_bridges.head()

,STATE_CODE_001,STRUCTURE_NUMBER_008,COUNTY_CODE_003,FEATURES_DESC_006A,FACILITY_CARRIED_007,LOCATION_009,LAT_016,LONG_017,MIN_VERT_CLR_010,DETOUR_KILOS_019,...,APPR_KIND_044A,APPR_TYPE_044B,DECK_STRUCTURE_TYPE_107,SURFACE_TYPE_108A,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,SUFFICIENCY_RATING_tar_yr
11,51,1VA3701,810.0,'Lake Smith Outlet ','Amphibious Drive ','0.KM NE of SR-166 ',36543290,76100580,99.99,2.0,...,0.0,0.0,3,0,0,0,5,5,5,72.7
12,51,1VA591A,810.0,'Canal ','Golf Course Maint ','Near 18th Hole ',36480234,76013348,99.99,4.0,...,0.0,0.0,1,0,0,0,5,5,6,52.3
13,51,2VA00AA,179.0,'Aquia Creek ','MCB-3 ','1.0KM SE of Camp Barrett ',38292730,77260090,99.99,22.0,...,0.0,0.0,2,3,0,0,5,7,5,69.2
14,51,2VA00CC,179.0,'Beaver Dam Run Spillway ','MCB-4 ','1.0KM North of MCB-3 ',38311950,77274620,99.99,6.0,...,0.0,0.0,1,6,0,0,5,6,6,63.6
15,51,2VA00DD,153.0,'Lucky Run ','MCB-8 ','10M West of MCB-1 ',38364580,77312120,99.99,12.0,...,0.0,0.0,2,6,0,0,7,6,5,79.1


## Data Types

In [23]:
# change numeric values to int or float types
for col in num_cols:
    df_bridges[col] = pd.to_numeric(df_bridges[col], errors='coerce')

In [24]:
# check features are numeric
df_bridges[num_cols].dtypes

MIN_VERT_CLR_010              float64
DETOUR_KILOS_019              float64
AGE                           float64
TRAFFIC_LANES_ON_028A         float64
TRAFFIC_LANES_UND_028B          int64
ADT_029                       float64
APPR_WIDTH_MT_032             float64
DEGREES_SKEW_034              float64
NAV_VERT_CLR_MT_039           float64
NAV_HORR_CLR_MT_040           float64
MAIN_UNIT_SPANS_045           float64
APPR_SPANS_046                float64
HORR_CLR_MT_047               float64
MAX_SPAN_LEN_MT_048           float64
STRUCTURE_LEN_MT_049          float64
LEFT_CURB_MT_050A             float64
RIGHT_CURB_MT_050B            float64
ROADWAY_WIDTH_MT_051          float64
DECK_WIDTH_MT_052             float64
VERT_CLR_OVER_MT_053          float64
VERT_CLR_UND_054B             float64
LAT_UND_MT_055B               float64
LEFT_LAT_UND_MT_056           float64
RECON_AGE                      object
PERCENT_ADT_TRUCK_109         float64
SUFFICIENCY_RATING_feat_yr    float64
dtype: objec

In [25]:
# change categorical values to string type
df_bridges[cat_cols] = df_bridges[cat_cols].astype(str)

In [26]:
# check categorical data types
df_bridges[cat_cols].dtypes

TOLL_020                   object
MAINTENANCE_021            object
FUNCTIONAL_CLASS_026       object
DESIGN_LOAD_031            object
MEDIAN_CODE_033            object
STRUCTURE_FLARED_035       object
RAILINGS_036A              object
TRANSITIONS_036B           object
APPR_RAIL_036C             object
APPR_RAIL_END_036D         object
HISTORY_037                object
NAVIGATION_038             object
OPEN_CLOSED_POSTED_041     object
SERVICE_ON_042A            object
SERVICE_UND_042B           object
STRUCTURE_KIND_043A        object
STRUCTURE_TYPE_043B        object
APPR_KIND_044A             object
APPR_TYPE_044B             object
DECK_STRUCTURE_TYPE_107    object
SURFACE_TYPE_108A          object
MEMBRANE_TYPE_108B         object
DECK_PROTECTION_108C       object
DECK_COND_058              object
SUPERSTRUCTURE_COND_059    object
SUBSTRUCTURE_COND_060      object
dtype: object

# NaNs

In [27]:
# check for NaN values in feature data
df_bridges[num_cols + cat_cols].isna().sum()

MIN_VERT_CLR_010                0
DETOUR_KILOS_019                0
AGE                             0
TRAFFIC_LANES_ON_028A           0
TRAFFIC_LANES_UND_028B          0
ADT_029                         0
APPR_WIDTH_MT_032               0
DEGREES_SKEW_034                0
NAV_VERT_CLR_MT_039             0
NAV_HORR_CLR_MT_040             0
MAIN_UNIT_SPANS_045             0
APPR_SPANS_046                  0
HORR_CLR_MT_047                 0
MAX_SPAN_LEN_MT_048             0
STRUCTURE_LEN_MT_049            0
LEFT_CURB_MT_050A               0
RIGHT_CURB_MT_050B              0
ROADWAY_WIDTH_MT_051            0
DECK_WIDTH_MT_052               0
VERT_CLR_OVER_MT_053            0
VERT_CLR_UND_054B               0
LAT_UND_MT_055B                 0
LEFT_LAT_UND_MT_056             0
RECON_AGE                       0
PERCENT_ADT_TRUCK_109         173
SUFFICIENCY_RATING_feat_yr      0
TOLL_020                        0
MAINTENANCE_021                 0
FUNCTIONAL_CLASS_026            0
DESIGN_LOAD_03

In [28]:
# check for NaN values in target data
df_bridges['SUFFICIENCY_RATING_tar_yr'].isna().sum()

0

## Impute Mean

In [29]:
# fill NaNs with mean value
df_bridges['PERCENT_ADT_TRUCK_109'].fillna(df_bridges['PERCENT_ADT_TRUCK_109'].mean(), inplace=True)

# Data Overview

In [30]:
df_bridges[num_cols + cat_cols].describe()

,MIN_VERT_CLR_010,DETOUR_KILOS_019,AGE,TRAFFIC_LANES_ON_028A,TRAFFIC_LANES_UND_028B,ADT_029,APPR_WIDTH_MT_032,DEGREES_SKEW_034,NAV_VERT_CLR_MT_039,NAV_HORR_CLR_MT_040,...,LEFT_CURB_MT_050A,RIGHT_CURB_MT_050B,ROADWAY_WIDTH_MT_051,DECK_WIDTH_MT_052,VERT_CLR_OVER_MT_053,VERT_CLR_UND_054B,LAT_UND_MT_055B,LEFT_LAT_UND_MT_056,PERCENT_ADT_TRUCK_109,SUFFICIENCY_RATING_feat_yr
count,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,...,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000,1639.000000
mean,98.647907,27.702257,52.727883,2.178157,0.782184,8298.545455,8.782001,13.001220,0.156681,0.437157,...,0.335937,0.345149,8.642099,9.885906,98.916571,1.377322,1.128920,2.551556,4.114598,69.718121
std,11.255159,54.419742,18.047567,1.038331,2.025967,19795.243965,5.725902,18.141453,1.531115,4.918875,...,0.471569,0.484030,5.312301,6.145489,9.950856,2.781888,5.800066,14.459409,5.428790,8.063368
min,0.000000,0.000000,8.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,3.000000,3.000000,3.220000,0.000000,0.000000,0.000000,0.000000,50.000000
25%,99.990000,3.000000,40.000000,2.000000,0.000000,289.000000,5.500000,0.000000,0.000000,0.000000,...,0.000000,0.000000,6.100000,6.700000,99.990000,0.000000,0.000000,0.000000,1.000000,64.100000
50%,99.990000,8.000000,49.000000,2.000000,0.000000,1427.000000,7.000000,0.000000,0.000000,0.000000,...,0.200000,0.200000,7.300000,8.000000,99.990000,0.000000,0.000000,0.000000,2.000000,71.200000
75%,99.990000,16.000000,72.000000,2.000000,0.000000,7022.000000,9.800000,28.000000,0.000000,0.000000,...,0.500000,0.500000,9.100000,10.700000,99.990000,0.000000,0.000000,0.000000,5.000000,77.000000
max,99.990000,199.000000,117.000000,13.000000,16.000000,186071.000000,64.600000,99.000000,23.100000,128.000000,...,3.000000,3.000000,67.400000,72.900000,99.990000,23.460000,99.900000,99.900000,93.000000,79.900000


# Export dataframe

In [31]:
# ID column for index
bridge_id = df_bridges['STATE_CODE_001'].astype(str) + \
'-' + df_bridges['STRUCTURE_NUMBER_008'].astype(str).apply(lambda x: x.strip())
# set index to unique structure id
df_bridges.set_index(bridge_id, inplace=True)
# drop columns used for id
df_bridges.drop(columns=['STATE_CODE_001', 'STRUCTURE_NUMBER_008'], inplace=True)
df_bridges.head()

,COUNTY_CODE_003,FEATURES_DESC_006A,FACILITY_CARRIED_007,LOCATION_009,LAT_016,LONG_017,MIN_VERT_CLR_010,DETOUR_KILOS_019,AGE,TRAFFIC_LANES_ON_028A,...,APPR_KIND_044A,APPR_TYPE_044B,DECK_STRUCTURE_TYPE_107,SURFACE_TYPE_108A,MEMBRANE_TYPE_108B,DECK_PROTECTION_108C,DECK_COND_058,SUPERSTRUCTURE_COND_059,SUBSTRUCTURE_COND_060,SUFFICIENCY_RATING_tar_yr
51-1VA3701,810.0,'Lake Smith Outlet ','Amphibious Drive ','0.KM NE of SR-166 ',36543290,76100580,99.99,2.0,53.0,2.0,...,0.0,0.0,3,0,0,0,5,5,5,72.7
51-1VA591A,810.0,'Canal ','Golf Course Maint ','Near 18th Hole ',36480234,76013348,99.99,4.0,42.0,2.0,...,0.0,0.0,1,0,0,0,5,5,6,52.3
51-2VA00AA,179.0,'Aquia Creek ','MCB-3 ','1.0KM SE of Camp Barrett ',38292730,77260090,99.99,22.0,60.0,2.0,...,0.0,0.0,2,3,0,0,5,7,5,69.2
51-2VA00CC,179.0,'Beaver Dam Run Spillway ','MCB-4 ','1.0KM North of MCB-3 ',38311950,77274620,99.99,6.0,60.0,2.0,...,0.0,0.0,1,6,0,0,5,6,6,63.6
51-2VA00DD,153.0,'Lucky Run ','MCB-8 ','10M West of MCB-1 ',38364580,77312120,99.99,12.0,60.0,2.0,...,0.0,0.0,2,6,0,0,7,6,5,79.1


In [33]:
# export ID, numeric, categorical and target dfs to csv files
df_bridges.iloc[:, :6].to_csv('data/bridges_id.csv')
df_bridges[num_cols].to_csv('data/bridges_num.csv')
df_bridges[cat_cols].to_csv('data/bridges_cat.csv')
df_bridges['SUFFICIENCY_RATING_tar_yr'].to_csv('data/bridges_tar.csv')